In [4]:
# General Imports

import numpy as np
import os
import time
from tqdm.notebook import tqdm

# Audio Imports

import librosa
from scipy.io import wavfile as wav
# import simpleaudio as sa
import IPython.display as ipd
import sounddevice as sd
# import wave

/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Users/kappa/opt/miniconda3/envs/dsim/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [ ]:
#attempt = 0
#name = "gian"
name = "khaled"
duration = 2
rec_rate = 8000 # sample rate of the provided recordings
recordings_dir = "my_recordings/"

In [ ]:
if not os.path.isdir(recordings_dir):
    os.makedirs(recordings_dir)

In [ ]:
def create_recordings(number_of_recordings, number):
    i=0
    while i < number_of_recordings:
        print(f"Recording number {number}")
        filename = recordings_dir+"_".join([str(number), name, str(i)])+".wav"
        print(f"\nAttempt: {i}")
        print("Ready in 3...", end = "")
        time.sleep(1)
        print("2...", end = "")
        time.sleep(1)
        print("1...")
        time.sleep(1)
        print("Go.")
        rec = sd.rec(int(duration * rec_rate), samplerate=rec_rate, channels=1, blocking=True)
        print("Playing the recording.")
        sd.play(rec, rec_rate)

        # after hearing the recording, decide whether to record it again or continue to next number
        # if you type anything, record again
        # if you press enter, save current recording & go to next number
        ok = input("OK?")
        if ok == "":
            librosa.output.write_wav(filename, rec, rec_rate)
            i += 1
        ipd.clear_output(wait=True)

In [ ]:
create_recordings(10, 0)

In [ ]:
create_recordings(10, 1)

In [ ]:
create_recordings(10, 2)

In [ ]:
create_recordings(10, 3)

In [ ]:
create_recordings(10, 4)

In [ ]:
create_recordings(10, 5)

In [ ]:
create_recordings(10, 6)

In [ ]:
create_recordings(10, 7)

In [ ]:
create_recordings(10, 8)

In [ ]:
create_recordings(10, 9)

# Remove silence from recordings

In [1]:
import subprocess

In [2]:
output_dir = "output/"
input_dir = "my_recordings/" # recordings_dir

In [5]:
# check if input directory exists
if not os.path.isdir(input_dir):
    print("There should be an input \"my recordings\" directory.")

# create output directory if not there yet
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

In [7]:
temp1 = output_dir+"temp1.wav"
temp2 = output_dir+"temp2.wav"
temp3 = output_dir+"temp3.wav"
db=-48 # default was -96, but it didn't cut anything; test with a few recordings before deciding
all_files = os.listdir(input_dir)

for file in tqdm(all_files):
    subprocess.run(["ffmpeg", "-y", "-i", input_dir+file, "-af", f"silenceremove=1:0:{db}dB", temp1])
    subprocess.run(["ffmpeg", "-y", "-i", temp1, "-af", "areverse", temp2])
    subprocess.run(["ffmpeg", "-y", "-i", temp2, "-af", f"silenceremove=1:0.1:{db}dB", temp3])
    subprocess.run(["ffmpeg", "-y", "-i", temp3, "-af", "areverse", output_dir+file])
    try:
        
        os.remove(temp1)
        os.remove(temp2)
        os.remove(temp3)
    c

FileNotFoundError: [Errno 2] No such file or directory: 'output/temp1.wav'

In [ ]:
# check the durations of the cut recordings
# (shold be less than 2 seconds)
durations = []
for file in tqdm(os.listdir("output")):
    sample, sample_rate = librosa.core.load('output/'+file)
    duration = sample.shape[0]/sample_rate
    durations.append(duration)
durations

In [ ]:
# test if the cut makes sense (i.e., if number can still be heard)
for rec_file in tqdm(os.listdir("output")):
    rec_rate, rec = wav.read("output/"+rec_file)
    sd.play(rec, rec_rate)
    time.sleep(1) # without this, the audio cannot be heard...